# Configuração geral para consulta em Views montadas em SQL scrips

## Bibliotecas Utilizadas

In [ ]:
#%%pyspark
#
######################################################################
# Checa Disponibilidade da Biblioteca 
######################################################################
#
try:
    import pymssql
except ModuleNotFoundError:
    %pip install --quiet pymssql
    import pymssql

import json

## Definições Gerais do Banco

In [ ]:
# Define o header com token para importação dos dados via api
ls_keyvault = mssparkutils.credentials.getPropertiesAll('LS_KEYVAULT_01')

converter_dic_kv = json.loads(ls_keyvault)

#coletando o endpoint
end_point_kv = (converter_dic_kv['Endpoint'].split("/"))[2]
kv_name = (end_point_kv.split("."))[0]

# password armazenada no Key Vaul do cofre kv-dtan-prd
psswd = mssparkutils.credentials.getSecret(kv_name,'KV-PWD-CI-DS-ISOLADO','LS_KEYVAULT_01')

#
#######################################################################
# Definições Gerais Do Banco
#######################################################################
#
dbServer   = "syn-dtan-csting-ondemand.sql.azuresynapse.net"
dbUser     = "dbadmin"
dbPassword = psswd
dbDataBase = "mercadopotencial"

## Função Geral de Consulta

In [1]:
def ConsultaView(vw_fonte):
    '''
        Função que retorna a tabela origem em PRD .parquet definina na VIEW
    
    Parâmetros:
    -----------
        vw_fonte: string
            Nome da view definida em SQL scrips
    Retorno:
    -----------
        dfResultado: pd.DataFrame
            Tabela com o resultado da consulta
    '''
    
    sqlCMD = "SELECT * FROM [CI].[" + vw_fonte + "]"

    dbConexao = pymssql.connect(server   = dbServer,                
                                user     = dbUser, 
                                password = dbPassword, 
                                database = dbDataBase)

    cursor = dbConexao.cursor(as_dict=True)
    cursor.execute(sqlCMD)
    
    dfResultado = pd.DataFrame(cursor.fetchall())

    dbConexao.close()
    
    return(dfResultado)

In [ ]:
print('Configurações executadas com sucesso!')
print('')
print('Função geral de consulta carregada.\nMaiores detalhes executar:\nhelp(ConsultaView)')